Carregando o CSV com os dados dos alunos referentes à 2016

import os
import pandas as pd
import numpy as np3

if not os.path.exists("data"):
    os.mkdir("data")
    !cd data && curl -O {"http://download.inep.gov.br/microdados/microdados_censo_superior_2016.zip"} && unzip {"'*.zip'"}
    !cd data/DADOS && unrar e "*.rar" && mv *.CSV ../
    !cd data && rm -r ANEXOS LEIA-ME DADOS FILTROS && rm microdados_censo_superior_2016.zip



Coloquei em markdown porque tava demorando pra baixar, já que eu tinha os arquivos aqui... preferi deixar assim por enquanto

In [25]:
import pandas as pd

dataframe_alunos_2016 =  pd.read_csv('data/DM_ALUNO.CSV',
                                     sep='|', encoding='latin1',
                                     usecols=['CO_IES','NO_IES' ,'CO_TURNO_ALUNO', 'CO_GRAU_ACADEMICO', 'CO_ALUNO_CURSO', 'DS_COR_RACA_ALUNO',
                                            'CO_MODALIDADE_ENSINO', 'NO_OCDE', 'DS_SEXO_ALUNO', 'NU_ANO_ALUNO_NASC', 'CO_ALUNO_SITUACAO',
                                             'IN_FINANC_ESTUDANTIL', 'CO_TIPO_ESCOLA_ENS_MEDIO', 'ANO_INGRESSO', 'IN_DEF_FISICA'])
dataframe_alunos_2016.head()

,CO_IES,NO_IES,CO_TURNO_ALUNO,CO_GRAU_ACADEMICO,CO_MODALIDADE_ENSINO,NO_OCDE,CO_ALUNO_CURSO,DS_COR_RACA_ALUNO,DS_SEXO_ALUNO,NU_ANO_ALUNO_NASC,IN_DEF_FISICA,CO_ALUNO_SITUACAO,IN_FINANC_ESTUDANTIL,CO_TIPO_ESCOLA_ENS_MEDIO,ANO_INGRESSO
0,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,1.0,1,Direito,25001314,Parda,Masculino,1987,NaN,2,NaN,1,2016
1,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,1.0,1,Direito,25037243,Parda,Masculino,1988,NaN,2,NaN,1,2012
2,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,1.0,1,Direito,25030916,Preta,Feminino,1988,NaN,3,NaN,1,2015
3,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,1.0,1,Direito,25016919,Parda,Masculino,1977,NaN,3,NaN,1,2013
4,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1.0,1.0,1,Direito,25001942,Parda,Masculino,1983,NaN,4,NaN,1,2011


print(dataframe_alunos_2016.groupby('IN_FINANC_ESTUDANTIL').count())

dataframe_alunos_2016.to_csv('data/alunos2016.csv')

In [26]:
import pandas as pd

DF_alunos2016 = dataframe_alunos_2016  #pd.read_csv('data/alunos2016.csv')
DF_alunos2016.head()

,CO_IES,NO_IES,CO_TURNO_ALUNO,CO_GRAU_ACADEMICO,CO_MODALIDADE_ENSINO,NO_OCDE,CO_ALUNO_CURSO,DS_COR_RACA_ALUNO,DS_SEXO_ALUNO,NU_ANO_ALUNO_NASC,IN_DEF_FISICA,CO_ALUNO_SITUACAO,IN_FINANC_ESTUDANTIL,CO_TIPO_ESCOLA_ENS_MEDIO,ANO_INGRESSO
0,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,1.0,1,Direito,25001314,Parda,Masculino,1987,NaN,2,NaN,1,2016
1,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,1.0,1,Direito,25037243,Parda,Masculino,1988,NaN,2,NaN,1,2012
2,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,1.0,1,Direito,25030916,Preta,Feminino,1988,NaN,3,NaN,1,2015
3,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,1.0,1,Direito,25016919,Parda,Masculino,1977,NaN,3,NaN,1,2013
4,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1.0,1.0,1,Direito,25001942,Parda,Masculino,1983,NaN,4,NaN,1,2011


In [27]:
qtd_cursos = len(DF_alunos2016.NO_OCDE.drop_duplicates())
qtd_cursos

350

In [28]:
curso_raca = DF_alunos2016
curso_raca = pd.melt(curso_raca, id_vars = 'NO_OCDE', value_vars ='DS_COR_RACA_ALUNO')

curso_raca.head()



,NO_OCDE,variable,value
0,Direito,DS_COR_RACA_ALUNO,Parda
1,Direito,DS_COR_RACA_ALUNO,Parda
2,Direito,DS_COR_RACA_ALUNO,Preta
3,Direito,DS_COR_RACA_ALUNO,Parda
4,Direito,DS_COR_RACA_ALUNO,Parda


In [29]:
curso_raca = curso_raca.rename(columns= {'value' : 'raca'})
curso_raca = curso_raca.rename(columns= {'NO_OCDE' : 'curso'})
curso_raca = curso_raca.rename(columns= {'variable' : 'qtd'})

curso_raca = curso_raca.replace(to_replace = 'DS_COR_RACA_ALUNO', value = 1)

curso_raca = curso_raca.pivot_table(index = 'curso', columns='raca', values= 'qtd',aggfunc='sum')

curso_raca.head()

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta
curso,,,,,,,
Acústica,4.0,NaN,187.0,NaN,NaN,19.0,10.0
Administração,345790.0,15172.0,393559.0,4783.0,34924.0,244571.0,54294.0
Administração de cooperativas,398.0,17.0,597.0,6.0,6.0,382.0,156.0
Administração de redes,11294.0,575.0,11277.0,198.0,924.0,8918.0,2618.0
Administração de sistemas de informações,NaN,NaN,NaN,NaN,NaN,6.0,NaN


In [30]:
curso_raca.loc['Administração']

raca
Aluno não quis declarar cor/raça    345790.0
Amarela                              15172.0
Branca                              393559.0
Indígena                              4783.0
Não dispõe da informação             34924.0
Parda                               244571.0
Preta                                54294.0
Name: Administração, dtype: float64

Com isso, podemos selecionar relações entre os cursos e as raças

In [31]:
for i in range(0,6):
    print(curso_raca.iloc[:,i].idxmax())



Administração
Direito
Direito
Direito
Administração
Direito


Pegando o curso com maior qtd por coluna

In [32]:
curso_raca.iloc[:].nlargest(5,'Amarela')

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta
curso,,,,,,,
Direito,311136.0,17151.0,456318.0,6425.0,23740.0,257764.0,62355.0
Administração,345790.0,15172.0,393559.0,4783.0,34924.0,244571.0,54294.0
Pedagogia,293977.0,10045.0,293059.0,5789.0,33826.0,220415.0,53091.0
Engenharia civil,135277.0,9478.0,202143.0,3574.0,7798.0,111778.0,25950.0
Ciências contábeis,163908.0,7583.0,174480.0,2634.0,11614.0,126821.0,27837.0


Dessa forma mostra os n maiores cursos por raça, mas "ordena" pela coluna selecionada, nesse caso, foi "Amarela"

In [33]:
DF_IES =  pd.read_csv('data/DM_IES.CSV', sep='|', encoding='latin1')
DF_IES.head()

,CO_IES,NO_IES,SGL_IES,CO_MANTENEDORA,NO_MANTENEDORA,CO_CATEGORIA_ADMINISTRATIVA,DS_CATEGORIA_ADMINISTRATIVA,CO_ORGANIZACAO_ACADEMICA,DS_ORGANIZACAO_ACADEMICA,CO_MUNICIPIO_IES,...,VL_RECEITA_PROPRIA,VL_TRANSFERENCIA,VL_OUTRA_RECEITA,VL_DES_PESSOAL_REM_DOCENTE,VL_DES_PESSOAL_REM_TECNICO,VL_DES_PESSOAL_ENCARGO,VL_DES_CUSTEIO,VL_DES_INVESTIMENTO,VL_DES_PESQUISA,VL_DES_OUTRAS
0,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT,1,FUNDACAO UNIVERSIDADE FEDERAL DE MATO GROSSO,1,Pública Federal,1,Universidade,5103403,...,7847404.77,8.463274e+08,0.000000e+00,2.409985e+08,1.047201e+08,1.016396e+08,1.301580e+08,17926971.06,14034108.54,2.446976e+08
1,2,UNIVERSIDADE DE BRASÍLIA,UNB,2,FUNDACAO UNIVERSIDADE DE BRASILIA,1,Pública Federal,1,Universidade,5300108,...,42952090.56,1.125740e+08,1.270250e+08,9.406867e+08,0.000000e+00,9.406867e+08,3.031958e+08,29754378.97,12466559.85,3.734387e+08
2,3,UNIVERSIDADE FEDERAL DE SERGIPE,UFS,3,FUNDACAO UNIVERSIDADE FEDERAL DE SERGIPE,1,Pública Federal,1,Universidade,2806701,...,4744562.76,7.149826e+08,2.693960e+08,2.817225e+08,1.516967e+08,1.128631e+08,1.558269e+08,53033112.85,25766649.67,0.000000e+00
3,4,UNIVERSIDADE FEDERAL DO AMAZONAS,UFAM,4,FUNDACAO UNIVERSIDADE DO AMAZONAS,1,Pública Federal,1,Universidade,1302603,...,7331203.25,6.319321e+08,7.760058e+05,3.123510e+08,0.000000e+00,9.795765e+07,1.474634e+08,31936025.81,1055524.04,1.368965e+08
4,6,UNIVERSIDADE FEDERAL DE OURO PRETO,UFOP,6,UNIVERSIDADE FEDERAL DE OURO PRETO,1,Pública Federal,1,Universidade,3146107,...,3746358.40,4.009708e+08,3.957849e+06,1.543418e+08,7.137004e+07,3.582467e+07,4.110826e+07,1676309.58,1142377.41,1.032126e+08


IES = [None] * len(DF_alunos2016)

i = 0
j = 0

while i < len(DF_IES):
    while j < len(auxAlunos):
        if (DF_IES.iloc[i,0] == auxAlunos.iloc[j,0]):
            IES[j] = DF_IES.iloc[i,2]
            print(IES[j],i,j)
            j = j+1
            
        else:
            auxAlunos = auxAlunos.drop(auxAlunos.index[0:j])
    i = i +1


In [34]:
IES_raca = DF_alunos2016
IES_raca = pd.melt(IES_raca, id_vars = 'NO_IES', value_vars ='DS_COR_RACA_ALUNO')

IES_raca.head()


,NO_IES,variable,value
0,UNIVERSIDADE FEDERAL DE MATO GROSSO,DS_COR_RACA_ALUNO,Parda
1,UNIVERSIDADE FEDERAL DE MATO GROSSO,DS_COR_RACA_ALUNO,Parda
2,UNIVERSIDADE FEDERAL DE MATO GROSSO,DS_COR_RACA_ALUNO,Preta
3,UNIVERSIDADE FEDERAL DE MATO GROSSO,DS_COR_RACA_ALUNO,Parda
4,UNIVERSIDADE FEDERAL DE MATO GROSSO,DS_COR_RACA_ALUNO,Parda


In [35]:
IES_raca = IES_raca.rename(columns= {'value' : 'raca'})
IES_raca = IES_raca.rename(columns= {'NO_IES' : 'IES'})
IES_raca = IES_raca.rename(columns= {'variable' : 'qtd'})

IES_raca = IES_raca.replace(to_replace = 'DS_COR_RACA_ALUNO', value = 1)

IES_raca = IES_raca.pivot_table(index = 'IES', columns='raca', values= 'qtd',aggfunc='sum')

IES_raca.head()

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta
IES,,,,,,,
ABEU - CENTRO UNIVERSITÁRIO,979.0,83.0,1822.0,18.0,440.0,2255.0,731.0
ALFA - Faculdade de Almenara,421.0,1.0,18.0,1.0,22.0,48.0,2.0
Academia Militar das Agulhas Negras,148.0,3.0,531.0,NaN,1.0,240.0,88.0
CASTELLI ESCOLA SUPERIOR DE HOTELARIA,43.0,1.0,19.0,NaN,1.0,3.0,NaN
CENTRO DE EDUCAÇÃO TÉCNICA DA UTRAMIG,35.0,NaN,9.0,1.0,NaN,15.0,1.0


In [36]:
IES_raca.iloc[:].nlargest(5,'Branca')

raca,Aluno não quis declarar cor/raça,Amarela,Branca,Indígena,Não dispõe da informação,Parda,Preta
IES,,,,,,,
CENTRO UNIVERSITÁRIO INTERNACIONAL,60134.0,2779.0,108121.0,498.0,NaN,44581.0,9440.0
UNIVERSIDADE NOVE DE JULHO,30178.0,4354.0,98590.0,1119.0,35.0,62463.0,21124.0
UNIVERSIDADE ANHANGUERA - UNIDERP,128875.0,3695.0,98387.0,938.0,526.0,59163.0,15128.0
UNIVERSIDADE PAULISTA,302081.0,3887.0,87319.0,918.0,291.0,139341.0,9084.0
UNIVERSIDADE ESTÁCIO DE SÁ,243843.0,9478.0,78759.0,2034.0,1476.0,45124.0,36690.0


In [37]:
IES_raca.loc['UNIVERSIDADE DE SÃO PAULO']

raca
Aluno não quis declarar cor/raça    18026.0
Amarela                              3604.0
Branca                              41553.0
Indígena                              106.0
Não dispõe da informação                3.0
Parda                                5325.0
Preta                                1482.0
Name: UNIVERSIDADE DE SÃO PAULO, dtype: float64

In [38]:
DF_alunos2016.isnull().sum()

CO_IES                             0
NO_IES                             0
CO_TURNO_ALUNO               2350313
CO_GRAU_ACADEMICO              60766
CO_MODALIDADE_ENSINO               0
NO_OCDE                        52270
CO_ALUNO_CURSO                     0
DS_COR_RACA_ALUNO                  0
DS_SEXO_ALUNO                      0
NU_ANO_ALUNO_NASC                  0
IN_DEF_FISICA               11399409
CO_ALUNO_SITUACAO                  0
IN_FINANC_ESTUDANTIL         2232187
CO_TIPO_ESCOLA_ENS_MEDIO           0
ANO_INGRESSO                       0
dtype: int64

In [44]:
defFisica = DF_alunos2016

defFisica = defFisica.dropna(subset=['IN_DEF_FISICA'])

In [46]:
defFisica.head()


,CO_IES,NO_IES,CO_TURNO_ALUNO,CO_GRAU_ACADEMICO,CO_MODALIDADE_ENSINO,NO_OCDE,CO_ALUNO_CURSO,DS_COR_RACA_ALUNO,DS_SEXO_ALUNO,NU_ANO_ALUNO_NASC,IN_DEF_FISICA,CO_ALUNO_SITUACAO,IN_FINANC_ESTUDANTIL,CO_TIPO_ESCOLA_ENS_MEDIO,ANO_INGRESSO
2726,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,1.0,1,Ciências contábeis,25057240,Branca,Masculino,1988,0.0,3,NaN,1,2014
5307,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,1.0,1.0,1,Administração,25004843,Parda,Masculino,1952,1.0,3,NaN,1,2015
26939,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,1.0,1,Administração dos serviços de saúde,25004869,Parda,Masculino,1978,1.0,3,NaN,1,2015
29757,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,3.0,2.0,1,Formação de professor de sociologia,25004907,Branca,Feminino,1969,0.0,4,NaN,0,2012
30268,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,4.0,2.0,1,Formação de professor de filosofia,25065086,Preta,Masculino,1974,1.0,2,NaN,1,2016
